# TFRS - Ranking model

In [26]:
# !pip install tensorflow-recommenders google-cloud-aiplatform --user
! python3 -c "import google.cloud.aiplatform; print('aiplatform SDK version: {}'.format(google.cloud.aiplatform.__version__))"

aiplatform SDK version: 1.26.1


## Load env config

In [27]:
# naming convention for all cloud resources
VERSION        = "v1"                  # TODO
PREFIX         = f'ndr-{VERSION}'      # TODO

print(f"PREFIX = {PREFIX}")

PREFIX = ndr-v1


In [28]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "ndr-v1"
VERSION                  = "v1"

APP                      = "sp"
MODEL_TYPE               = "2tower"
FRAMEWORK                = "tfrs"
DATA_VERSION             = "v1"
TRACK_HISTORY            = "5"

BUCKET_NAME              = "ndr-v1-hybrid-vertex-bucket"
BUCKET_URI               = "gs://ndr-v1-hybrid-vertex-bucket"
SOURCE_BUCKET            = "spotify-million-playlist-dataset"

DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://ndr-v1-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

CANDIDATE_PREFIX         = "candidates"
TRAIN_DIR_PREFIX      

## Imports

In [29]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['TF_GPU_THREAD_MODE']='gpu_private'
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["CLOUD_ML_PROJECT_ID"] = PROJECT_ID

In [30]:
import json
import numpy as np
import pickle as pkl
import logging
import time
from pprint import pprint

# tensorflow
import tensorflow as tf
import tensorflow_recommenders as tfrs
# from tensorboard.plugins import projector

# Google Cloud SDKs
from google.cloud import aiplatform as vertex_ai
from google.cloud import storage

# GCS SDK client
storage_client = storage.Client(project=PROJECT_ID)

# ranking models
from util import feature_set_utils as feature_utils
from src.ranking import tf_ranking_model as tfrm

# from src.ranking import build_audio_ranker_v1 as build_audio_ranker
from src.ranking import train_utils

# feature_sets

# GPU
import gc
from numba import cuda

In [33]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)
    
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [35]:
device = cuda.get_current_device()
device.reset()
gc.collect()

37

## Dataset

In [36]:
!tree src/ranking

src/ranking
├── __init__.py
├── __pycache__
│   ├── __init__.cpython-37.pyc
│   ├── build_audio_ranker.cpython-37.pyc
│   ├── tf_ranking_model.cpython-37.pyc
│   ├── train_config.cpython-37.pyc
│   └── train_utils.cpython-37.pyc
├── build_audio_ranker.py
├── feature_sets.py
├── requirements.txt
├── task.py
├── tf_ranking_model.py
├── train_config.py
└── train_utils.py

1 directory, 13 files


### create dataset objects

In [38]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.AUTO

batch_size = 1024 #*16

#### train dataset

In [39]:
# train_files = []

# for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'data/{DATA_VERSION}/{TRAIN_DIR_PREFIX}/'):
#     if '.tfrecords' in blob.name:
#         train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))


# train_dataset = tf.data.Dataset.from_tensor_slices(train_files).prefetch(tf.data.AUTOTUNE)

# # ===================================================
# # audio feats ranker
# # ===================================================
# train_dataset = train_dataset.interleave(
#     train_utils.full_parse,
#     num_parallel_calls=tf.data.AUTOTUNE,
#     cycle_length=tf.data.AUTOTUNE, 
#     deterministic=False,
# ).map(
#     feature_utils.parse_rank_tfrecord, 
#     num_parallel_calls=tf.data.AUTOTUNE
# ).batch(
#     batch_size
# ).prefetch(
#     tf.data.AUTOTUNE,
# ).with_options(
#     options
# )

# ===================================================
# cache dataset
# ===================================================
# train_dataset = train_dataset.cache()
# train_dataset

In [40]:
# for x in train_dataset.batch(1).take(1):
#     pprint(len(x))
#     pprint(x)

#### validation set

In [41]:
valid_files = []

for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'data/{DATA_VERSION}/{VALID_DIR_PREFIX}/'):
    if '.tfrecords' in blob.name:
        valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

valid_dataset = tf.data.Dataset.from_tensor_slices(valid_files).prefetch(tf.data.AUTOTUNE)

# ===================================================
# baseline ranker feats
# ===================================================
# valid_dataset = valid_dataset.interleave(
#     train_utils.full_parse,
#     num_parallel_calls=tf.data.AUTOTUNE,
#     cycle_length=tf.data.AUTOTUNE, 
#     deterministic=False,
# ).map(
#     feature_sets.parse_tfrecord, 
#     num_parallel_calls=tf.data.AUTOTUNE
# ).batch(
#     batch_size
# ).prefetch(
#     tf.data.AUTOTUNE,
# ).with_options(
#     options
# )

# ===================================================
# audio feats ranker
# ===================================================
valid_dataset = valid_dataset.interleave(
    train_utils.full_parse,
    num_parallel_calls=tf.data.AUTOTUNE,
    cycle_length=tf.data.AUTOTUNE, 
    deterministic=False,
).map(
    feature_utils.parse_rank_tfrecord, #parse_rank_tfrecord, 
    num_parallel_calls=tf.data.AUTOTUNE
).batch(
    batch_size
).prefetch(
    tf.data.AUTOTUNE,
).with_options(
    options
)

# ===================================================
# cache dataset
# ===================================================
# valid_dataset = valid_dataset.cache() #1gb machine mem + 400 MB in candidate ds (src/two-tower.py)
# valid_dataset

TypeError: in user code:

    File "/home/jupyter/jw-repo2/spotify_mpd_two_tower/src/ranking/train_utils.py", line 107, in full_parse  *
        data = tf.data.TFRecordDataset(data)

    TypeError: The `filenames` argument must contain `tf.string` elements. Got `tf.float32` elements.


In [13]:
for x in valid_dataset.batch(1).take(1):
    # pprint(x)
    pprint(len(x))

53


### vocab dictionary

In [14]:
filehandler = open('vocab_dict.pkl', 'rb')
VOCAB_DICT = pkl.load(filehandler)
filehandler.close()

In [15]:
for keys in VOCAB_DICT:
    print(keys)

pl_name_src
track_name_pl
artist_name_pl
album_name_pl
artist_genres_pl
tracks_playlist_titles_pl
track_name_can
artist_name_can
album_name_can
artist_genres_can
track_pl_titles_can


## Setup Vertex Experiment

In [16]:
EXPERIMENT_NAME = f'local-ranker-v1'

invoke_time = time.strftime("%Y%m%d-%H%M%S")
RUN_NAME = f'run-{invoke_time}'

LOG_DIR = f"gs://{BUCKET_NAME}/{EXPERIMENT_NAME}/{RUN_NAME}/tb-logs/"

vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    experiment=EXPERIMENT_NAME
)

print(f"RUN_NAME: {RUN_NAME}")
print(f"LOG_DIR: {LOG_DIR}")

RUN_NAME: run-20230925-163646
LOG_DIR: gs://ndr-v1-hybrid-vertex-bucket/local-ranker-v1/run-20230925-163646/tb-logs/


### setup Tensorboard callbacks

Setup tensorboard below so training is visible and we can inspect the graph

TensorBoard Callback [API Reference](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard)

> *Note:* While profiling does not work for managed Tensorboard at this time, you can inspect the profiler with an [inline Tensorboard in another notebook](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks). You may be prompted to install the tensorflow profiler library

In [17]:
# use existing TB instance
# TB_RESOURCE_NAME = 'projects/934903580331/locations/us-central1/tensorboards/472921941339013120'

# # create new TB instance
TENSORBOARD_DISPLAY_NAME=f"{EXPERIMENT_NAME}"
tensorboard = vertex_ai.Tensorboard.create(display_name=TENSORBOARD_DISPLAY_NAME)
TB_RESOURCE_NAME = tensorboard.resource_name


print(f"TB_RESOURCE_NAME: {TB_RESOURCE_NAME}")

Creating Tensorboard
Create Tensorboard backing LRO: projects/934903580331/locations/us-central1/tensorboards/5899003028320550912/operations/5118567132304506880
Tensorboard created. Resource name: projects/934903580331/locations/us-central1/tensorboards/5899003028320550912
To use this Tensorboard in another session:
tb = aiplatform.Tensorboard('projects/934903580331/locations/us-central1/tensorboards/5899003028320550912')
TB_RESOURCE_NAME: projects/934903580331/locations/us-central1/tensorboards/5899003028320550912


## Build model

In [18]:
from src.ranking import build_audio_ranker as build_audio_ranker

In [19]:
# USE_CROSS_LAYER = True

SEED = 1234
MAX_PLAYLIST_LENGTH = 5
EMBEDDING_DIM = 128  
PROJECTION_DIM = int(EMBEDDING_DIM / 4) # 50 

USE_DROPOUT = True
DROPOUT_RATE = 0.33
MAX_TOKENS = 20000

LAYER_SIZES=[256,64]

LR = .1
opt = tf.keras.optimizers.Adagrad(LR)

In [20]:
# ===================================================
# baseline ranker feats
# ===================================================
# model = tfrm.TheRankingModel(
#     layer_sizes=LAYER_SIZES
#     , vocab_dict = VOCAB_DICT
#     , embedding_dim = EMBEDDING_DIM
#     , projection_dim = PROJECTION_DIM
#     , seed = SEED
#     , use_dropout = USE_DROPOUT
#     , dropout_rate = DROPOUT_RATE
#     , max_tokens = MAX_TOKENS
# )

# ===================================================
# audio feats ranker
# ===================================================
model = build_audio_ranker.TheRankingModel(
    layer_sizes=LAYER_SIZES
    , vocab_dict = VOCAB_DICT
    , embedding_dim = EMBEDDING_DIM
    , projection_dim = PROJECTION_DIM
    , seed = SEED
    , use_dropout = USE_DROPOUT
    , dropout_rate = DROPOUT_RATE
    , max_tokens = MAX_TOKENS
)

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [22]:
# model.summary()

In [23]:
# model.ranking_model.summary()

In [25]:
# model.fit(valid_dataset, epochs=1) # valid_dataset | train_dataset

### Inspect layers

In [33]:
## Quick look at the layers
print("Ranking model:")

for i, l in enumerate(model.ranking_model.layers):
    print(i, l.name)

Ranking model:
0 pl_name_src_text_embedding
1 track_name_pl_emb_model
2 track_danceability_pl_emb_model
3 track_energy_pl_emb_model
4 track_key_pl_emb_model
5 track_loudness_pl_emb_model
6 track_mode_pl_emb_model
7 track_speechiness_pl_emb_model
8 track_acousticness_pl_emb_model
9 track_instrumentalness_pl_emb_model
10 track_liveness_pl_emb_model
11 track_valence_pl_emb_model
12 track_tempo_pl_emb_model
13 time_signature_pl_emb_model
14 track_name_can_emb_model
15 artist_genres_can_emb_model
16 track_danceability_can_emb_model
17 track_energy_can_emb_model
18 track_key_can_emb_model
19 track_loudness_can_emb_model
20 track_mode_can_emb_model
21 track_speechiness_can_emb_model
22 track_acousticness_can_emb_model
23 track_instrumentalness_can_emb_model
24 track_liveness_can_emb_model
25 track_valence_can_emb_model
26 track_tempo_can_emb_model
27 time_signature_can_emb_model
28 rank_dense_layers


## Train ranking model (local)

In [34]:
NUM_EPOCHS = 3
VALID_FREQUENCY = 5
HIST_FREQ = 0
EMBED_FREQ = 0

LOCAL_TRAIN_DIR = f"local_train_dir/{EXPERIMENT_NAME}/{RUN_NAME}/"
LOCAL_CHECKPOINT_DIR = f"{LOCAL_TRAIN_DIR}/chkpts" # my_model.ckpt
LOCAL_EMB_FILE = f'{LOCAL_TRAIN_DIR}/embs/metadata.tsv'

In [35]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=LOCAL_TRAIN_DIR, 
    histogram_freq=HIST_FREQ, 
    write_graph=True,
    embeddings_freq=EMBED_FREQ,
    embeddings_metadata=LOCAL_EMB_FILE
    
    
        # profile_batch=(20,50) #run profiler on steps 20-40 - enable this line if you want to run profiler from the utils/ notebook
    )

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=LOCAL_CHECKPOINT_DIR + "/cp-{epoch:03d}-loss={loss:.2f}.ckpt", # cp-{epoch:04d}.ckpt" cp-{epoch:04d}.ckpt"
    save_weights_only=True,
    save_best_only=True,
    monitor='total_loss',
    mode='min',
    save_freq='epoch',
    verbose=1,
)

#start the timer and training
start_time = time.time()

layer_history = model.fit(
    valid_dataset,
    validation_data=valid_dataset,
    validation_freq=VALID_FREQUENCY,
    epochs=NUM_EPOCHS,
    steps_per_epoch=50,
    validation_steps=100,
    callbacks=[
        tensorboard_callback, 
        # train_utils.UploadTBLogsBatchEnd(
        #     log_dir=LOG_DIR, 
        #     experiment_name=EXPERIMENT_NAME, 
        #     tb_resource_name=TB_RESOURCE_NAME
        # ),
        model_checkpoint_callback
    ], 
    verbose=1
)

end_time = time.time()
val_keys = [v for v in layer_history.history.keys()]
runtime_mins = int((end_time - start_time) / 60)

Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node 'ranking/mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/conda/lib/pyt

### log Vertex Experiment

In [ ]:
vertex_ai.start_run(RUN_NAME, tensorboard=TB_RESOURCE_NAME)

vertex_ai.log_params(
    {
        "layers": str(LAYER_SIZES), 
        "learning_rate": LR,
        "num_epochs": NUM_EPOCHS,
        "batch_size": batch_size,
        "valid_freq": VALID_FREQUENCY,
    }
)

# gather the metrics for the last epoch to be saved in metrics
metrics_dict = {"train-time-minutes": runtime_mins}

_ = [metrics_dict.update({key: layer_history.history[key][-1]}) for key in val_keys]

vertex_ai.log_metrics(metrics_dict)

vertex_ai.end_run()

In [23]:
eval_dict = model.evaluate(valid_dataset, return_dict=True)
eval_dict

647/647 [==============================] - 73s 111ms/step - rmse_metric: 0.0021 - loss: 4.3395e-06 - regularization_loss: 0.0000e+00 - total_loss: 4.3395e-06


{'rmse_metric': 0.0020842566154897213,
 'loss': 2.216394932474941e-06,
 'regularization_loss': 0,
 'total_loss': 2.216394932474941e-06}

## save trained model

In [25]:
OUTPUT_BUCKET = 'gs://jt-tfrs-central-v4'
! gsutil mb -l $REGION $OUTPUT_BUCKET

Creating gs://jt-tfrs-central-v4/...


In [26]:
model_dir = f"{OUTPUT_BUCKET}/local-test-v2"

print(model_dir)

gs://jt-tfrs-central-v4/local-test-v1


In [27]:
tf.saved_model.save(model, export_dir=model_dir)

INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fa9fd0>, 139649424417872), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fa9fd0>, 139649424417872), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f021731ea90>, 139649422462768), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f021731ea90>, 139649422462768), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 128]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f021733d1d0>, 139648235291152), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 1), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fd6f10>, 139647318721280), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fd7350>, 139647318721920), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f0216fd7350>, 139647318721920), {}).


INFO:tensorflow:Assets written to: gs://jt-tfrs-central-v4/local-test-v1/assets


INFO:tensorflow:Assets written to: gs://jt-tfrs-central-v4/local-test-v1/assets


### load saved model

In [28]:
loaded = tf.saved_model.load(model_dir)

In [29]:
loaded

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f01c5e98a90>

In [27]:
test_uri = 'gs://jt-tfrs-central-v4/test-rank-src-v5/run-20230629-083212/model-dir/ranking_model'
loaded_v2 = tf.saved_model.load(test_uri)

In [28]:
loaded_v2

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f74f5686ad0>

# Matching Engine - tests

In [52]:
from google.cloud import aiplatform as vertex_ai

# Vertex SDK 
vertex_ai.init(project=PROJECT_ID, location=REGION)

import time

In [53]:
INDEX_ENDPOINT_URI = 'projects/934903580331/locations/us-central1/indexEndpoints/4438886771006111744' # ann 50e

In [54]:
ME_index_endpoint = vertex_ai.MatchingEngineIndexEndpoint(INDEX_ENDPOINT_URI)
ME_index_endpoint

resource name: projects/934903580331/locations/us-central1/indexEndpoints/4438886771006111744

In [55]:
DEPLOYED_INDEX_ID = ME_index_endpoint.deployed_indexes[0].id
print(f"DEPLOYED_INDEX_ID: {DEPLOYED_INDEX_ID}")

ME_index_endpoint.deployed_indexes

DEPLOYED_INDEX_ID: deployedann_e2e_v1


[id: "deployedann_e2e_v1"
index: "projects/934903580331/locations/us-central1/indexes/68706282596466688"
create_time {
  seconds: 1681232232
  nanos: 385340000
}
private_endpoints {
  match_grpc_address: "10.41.2.5"
}
index_sync_time {
  seconds: 1687983053
  nanos: 643665000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

In [56]:
ENDPOINT_URI = 'projects/934903580331/locations/us-central1/endpoints/3302875755234459648'
model_endpoint = vertex_ai.Endpoint(ENDPOINT_URI)
model_endpoint

resource name: projects/934903580331/locations/us-central1/endpoints/3302875755234459648

In [57]:
print(model_endpoint.gca_resource.deployed_models[0])

id: "4612884486101663744"
model: "projects/934903580331/locations/us-central1/models/8584575572326219776"
display_name: "query-tower-sp-2tower-tfrs-e2e-v1-pipev4"
create_time {
  seconds: 1681222077
  nanos: 256958000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-16"
  }
  min_replica_count: 1
  max_replica_count: 1
}
service_account: "notebooksa@hybrid-vertex.iam.gserviceaccount.com"
model_version_id: "1"



In [58]:
# # MAX_PLAYLIST_LENGTH --> 5
TEST_INSTANCE_5 = test_instances.TEST_INSTANCE_5
TEST_INSTANCE_5

{'album_name_can': 'Capoeira Electronica',
 'album_name_pl': ['Odilara',
  'Capoeira Electronica',
  'Capoeira Ultimate',
  'Festa Popular',
  'Capoeira Electronica'],
 'album_uri_can': 'spotify:album:2FsSSHGt8JM0JgRy6ZX3kR',
 'album_uri_pl': ['spotify:album:4Y8RfvZzCiApBCIZswj9Ry',
  'spotify:album:2FsSSHGt8JM0JgRy6ZX3kR',
  'spotify:album:55HHBqZ2SefPeaENOgWxYK',
  'spotify:album:150L1V6UUT7fGUI3PbxpkE',
  'spotify:album:2FsSSHGt8JM0JgRy6ZX3kR'],
 'artist_followers_can': 5170.0,
 'artist_genres_can': 'capoeira',
 'artist_genres_pl': ['samba moderno',
  'capoeira',
  'capoeira',
  'NONE',
  'capoeira'],
 'artist_name_can': 'Capoeira Experience',
 'artist_name_pl': ['Odilara',
  'Capoeira Experience',
  'Denis Porto',
  'Zambe',
  'Capoeira Experience'],
 'artist_pop_can': 24.0,
 'artist_pop_pl': [4.0, 24.0, 2.0, 0.0, 24.0],
 'artist_uri_can': 'spotify:artist:5SKEXbgzIdRl3gQJ23CnUP',
 'artist_uri_pl': ['spotify:artist:72oameojLOPWYB7nB8rl6c',
  'spotify:artist:5SKEXbgzIdRl3gQJ23CnUP',


In [60]:
playlist_emb = model_endpoint.predict([TEST_INSTANCE_5])
playlist_emb

Prediction(predictions=[[2.13599348, 0.665572762, 0.306572795, -0.642993093, 0.905611515, -0.451913774, -1.04571533, -1.47509861, 0.667647898, 1.392694, -0.580560744, 2.1635685, 1.15477574, 0.890988827, 0.987143219, -0.344569176, -0.727035463, -0.337408423, -1.34015453, -1.03236663, -0.695001364, -0.106723189, 0.0395636633, 1.66761637, 0.981151044, 0.879824638, 0.642675459, 2.18829846, -1.37034965, 0.116360746, -0.356807619, -0.00551904133, -1.61825109, 1.00126624, -0.543757379, -0.640756607, 0.584731758, -1.26177561, 1.33843577, 0.260981917, 0.691716552, 0.555235922, 0.835755944, 0.242919102, 0.508965909, -0.551774085, -0.752674818, 1.07970881, -0.278400809, 1.20874846, 0.634759903, 0.753106713, -0.331600159, -0.451509297, -0.44077149, -0.706078768, 0.518127501, 0.231186, 1.1358285, -0.811849892, -0.790187061, 0.0396273322, 0.101898566, -1.96892595, 0.238233238, -0.289123625, 0.353820384, 0.371209621, 1.90042961, -0.360776, 0.874980628, 0.49637109, -0.448792934, -0.249126494, -1.58931

In [68]:
len(playlist_emb.predictions[0])

128

In [72]:
import numpy as np 

original = playlist_emb.predictions[0]
opposite = (original * np.ones(128) * -1)
opposite

array([-2.13599348, -0.66557276, -0.3065728 ,  0.64299309, -0.90561152,
        0.45191377,  1.04571533,  1.47509861, -0.6676479 , -1.392694  ,
        0.58056074, -2.1635685 , -1.15477574, -0.89098883, -0.98714322,
        0.34456918,  0.72703546,  0.33740842,  1.34015453,  1.03236663,
        0.69500136,  0.10672319, -0.03956366, -1.66761637, -0.98115104,
       -0.87982464, -0.64267546, -2.18829846,  1.37034965, -0.11636075,
        0.35680762,  0.00551904,  1.61825109, -1.00126624,  0.54375738,
        0.64075661, -0.58473176,  1.26177561, -1.33843577, -0.26098192,
       -0.69171655, -0.55523592, -0.83575594, -0.2429191 , -0.50896591,
        0.55177409,  0.75267482, -1.07970881,  0.27840081, -1.20874846,
       -0.6347599 , -0.75310671,  0.33160016,  0.4515093 ,  0.44077149,
        0.70607877, -0.5181275 , -0.231186  , -1.1358285 ,  0.81184989,
        0.79018706, -0.03962733, -0.10189857,  1.96892595, -0.23823324,
        0.28912362, -0.35382038, -0.37120962, -1.90042961,  0.36

In [73]:
original

[2.13599348,
 0.665572762,
 0.306572795,
 -0.642993093,
 0.905611515,
 -0.451913774,
 -1.04571533,
 -1.47509861,
 0.667647898,
 1.392694,
 -0.580560744,
 2.1635685,
 1.15477574,
 0.890988827,
 0.987143219,
 -0.344569176,
 -0.727035463,
 -0.337408423,
 -1.34015453,
 -1.03236663,
 -0.695001364,
 -0.106723189,
 0.0395636633,
 1.66761637,
 0.981151044,
 0.879824638,
 0.642675459,
 2.18829846,
 -1.37034965,
 0.116360746,
 -0.356807619,
 -0.00551904133,
 -1.61825109,
 1.00126624,
 -0.543757379,
 -0.640756607,
 0.584731758,
 -1.26177561,
 1.33843577,
 0.260981917,
 0.691716552,
 0.555235922,
 0.835755944,
 0.242919102,
 0.508965909,
 -0.551774085,
 -0.752674818,
 1.07970881,
 -0.278400809,
 1.20874846,
 0.634759903,
 0.753106713,
 -0.331600159,
 -0.451509297,
 -0.44077149,
 -0.706078768,
 0.518127501,
 0.231186,
 1.1358285,
 -0.811849892,
 -0.790187061,
 0.0396273322,
 0.101898566,
 -1.96892595,
 0.238233238,
 -0.289123625,
 0.353820384,
 0.371209621,
 1.90042961,
 -0.360776,
 0.874980628,
 0

In [74]:
DEPLOYED_ANN_INDEX_ID = 'deployedann_e2e_v1'

In [78]:
ANN_response = ME_index_endpoint.match(
    deployed_index_id=DEPLOYED_ANN_INDEX_ID,
    queries=[original],
    num_neighbors=10
)
# ANN_response

In [79]:
ANN_response

[[MatchNeighbor(id='spotify:track:5ujxgPve1zpywKhdhprjd7', distance=-68.44378662109375),
  MatchNeighbor(id='spotify:track:4IkKAbJxc6XMt6dP0Wxn9q', distance=-69.26606750488281),
  MatchNeighbor(id='spotify:track:77sA6Lhq1aHyeiDn0Ezynk', distance=-69.32096862792969),
  MatchNeighbor(id='spotify:track:3oTZEx1KVuCaGnDwOaluRv', distance=-69.49063873291016),
  MatchNeighbor(id='spotify:track:2GclgkNkaCOgauDyGSgn5O', distance=-69.74806213378906),
  MatchNeighbor(id='spotify:track:4djyU5pTSNYq2ZuBFRia7I', distance=-69.94599914550781),
  MatchNeighbor(id='spotify:track:0nhk50UU12qwXp3ap9KZH5', distance=-70.12556457519531),
  MatchNeighbor(id='spotify:track:6wL1b6VtUdmddjXzpURDBr', distance=-70.27442932128906),
  MatchNeighbor(id='spotify:track:7oKRbhVq8AqvnfIrSmh6bN', distance=-70.29933166503906),
  MatchNeighbor(id='spotify:track:2wb8SJRrCx4k7bgTk0HPXU', distance=-70.32716369628906)]]

In [80]:
ANN_response_v2 = ME_index_endpoint.match(
    deployed_index_id=DEPLOYED_ANN_INDEX_ID,
    queries=[opposite],
    num_neighbors=10
)
# ANN_response

In [81]:
ANN_response_v2

[[MatchNeighbor(id='spotify:track:2vsNqssAT6DwTr8XMtkvuI', distance=120.07003784179688),
  MatchNeighbor(id='spotify:track:6FrqntvCqYk6pTloZ5NGcS', distance=119.92662811279297),
  MatchNeighbor(id='spotify:track:1GzygBcsXQW5HVEvIky76a', distance=118.85808563232422),
  MatchNeighbor(id='spotify:track:0rcB5TG1EeMRkZFJDkDtGF', distance=118.80642700195312),
  MatchNeighbor(id='spotify:track:4nr2whZo0rB6blw1BaacZz', distance=117.85485076904297),
  MatchNeighbor(id='spotify:track:2XIuiCqPKof2hJfNzLJ9uW', distance=117.66204833984375),
  MatchNeighbor(id='spotify:track:20EFdLOP1Rm37ha8BLbOYe', distance=117.60924530029297),
  MatchNeighbor(id='spotify:track:0atv7V8rRNHYY9hroCQddh', distance=117.54971313476562),
  MatchNeighbor(id='spotify:track:5JELaD59SYjlNvl47UggH3', distance=117.53295135498047),
  MatchNeighbor(id='spotify:track:5WAyXO575REnpzp56Lm3m9', distance=117.46258544921875)]]